In [12]:
!pip3 install xgboost

In [23]:
import time

import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from datetime import datetime, timedelta
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, KBinsDiscretizer, PolynomialFeatures

In [24]:
df_2017 = pd.read_csv('/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/Process_Mining_1/Sprint_2/BPI_2017_rfc_xgboost.csv')

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_98046/2845421153.py:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv('/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/Process_Mining_1/Sprint_2/BPI_2017_rfc_xgboost.csv')


In [25]:
df_2017

,Unnamed: 0.1,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,timestamp_difference,event_seq,time_to_next_event,time_to_next_event_seconds,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,concept:name next,concept:name next rfc
0,3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,...,0 days 00:01:20.618000,4,0 days 00:00:00.011000,0.01,35.17,0 days 00:01:21.088000,5 days 16:10:11.014999397,W_Handle leads,W_Complete application,W_Complete application
1,4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,...,0 days 00:00:00.011000,5,0 days 00:00:00.010000,0.01,0.01,0 days 00:01:21.099000,5 days 08:22:12.629814592,W_Complete application,A_Concept,A_Concept
2,5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,...,0 days 00:00:00.010000,6,4 days 23:34:08.079000,430448.08,179447.74,0 days 00:01:21.109000,5 days 05:51:35.263530324,A_Concept,A_Create Application,A_Create Application
3,6,6,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,...,0 days 00:23:35.087000,1,0 days 00:00:00.049000,0.05,0.04,0 days 00:00:00,4 days 18:38:55.010316100,A_Create Application,A_Submitted,A_Submitted
4,7,7,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,...,0 days 00:00:00.049000,2,0 days 00:00:00.191000,0.19,0.30,0 days 00:00:00.049000,4 days 18:50:11.306445745,A_Submitted,W_Handle leads,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201081,1201084,1201084,statechange,User_99,A_Pending,Application,ApplState_1437687321,complete,2017-02-01 09:58:43.970,Remaining debt home,...,0 days 00:00:00.005000,72,0 days 00:00:00.002000,0.00,0.00,75 days 14:18:05.736000,19 days 15:02:47.018194346,W_Call incomplete files,W_Validate application,W_Validate application
1201082,1201085,1201085,Deleted,User_99,W_Validate application,Workflow,Workitem_1690791617,ate_abort,2017-02-01 09:58:43.972,Remaining debt home,...,0 days 00:00:00.002000,73,0 days 00:00:00.009000,0.01,0.02,75 days 14:18:05.738000,20 days 05:59:20.038348289,W_Call incomplete files,O_Cancelled,W_Validate application
1201083,1201086,1201086,statechange,User_99,O_Cancelled,Offer,OfferState_2080870299,complete,2017-02-01 09:58:43.981,Remaining debt home,...,0 days 00:00:00.009000,74,0 days 00:00:00.007000,0.01,0.01,75 days 14:18:05.747000,20 days 07:06:36.925158626,W_Call incomplete files,O_Cancelled,W_Validate application
1201084,1201087,1201087,statechange,User_99,O_Cancelled,Offer,OfferState_1140159487,complete,2017-02-01 09:58:43.988,Remaining debt home,...,0 days 00:00:00.007000,75,0 days 00:00:00.006000,0.01,0.01,75 days 14:18:05.754000,20 days 04:04:09.814735216,W_Call incomplete files,O_Cancelled,W_Validate application


### finding columns for features

In [27]:
num_columns = df_2017.select_dtypes(include=[np.number]).columns
num_imputer = SimpleImputer(strategy='median')
df_2017[num_columns] = num_imputer.fit_transform(df_2017[num_columns])

# Impute missing values for categorical columns
cat_columns = df_2017.select_dtypes(include=['object', 'category']).columns
cat_imputer = SimpleImputer(strategy='most_frequent', fill_value='missing')
df_2017[cat_columns] = cat_imputer.fit_transform(df_2017[cat_columns])

# Encoding categorical variables
label_encoders = {}
le = LabelEncoder()
for column in cat_columns:
    df_2017[column] = le.fit_transform(df_2017[column])
    label_encoders[column] = le

# Calculate correlation for numerical features
correlations = df_2017[num_columns].corrwith(df_2017['time_to_next_event_seconds']).sort_values(ascending=False)
print("Correlations with the target:\n", correlations)

# Prepare df_2017 for mutual information calculation
# Drop the target variable and ensure no NaN values exist
X = df_2017.drop(columns=['time_to_next_event_seconds'])
y = df_2017['time_to_next_event_seconds']

# Calculate mutual information
mi_scores = mutual_info_regression(X, y, random_state=0)
mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
print("\nMutual Information Scores:\n", mi_scores)


Correlations with the target:
 time_to_next_event_seconds              1.000000
predicted_time_to_next_event_seconds    0.240850
Action                                  0.068948
lifecycle:transition                    0.054327
case:ApplicationType                    0.011768
timestamp_difference                    0.005181
case:RequestedAmount                    0.004244
Selected                                0.002347
Accepted                                0.001820
case:concept:name                      -0.000371
NumberOfTerms                          -0.000543
MonthlyCost                            -0.000624
OfferedAmount                          -0.000823
FirstWithdrawalAmount                  -0.000990
CreditScore                            -0.001973
event_seq                              -0.004017
case:LoanGoal                          -0.005607
org:resource                           -0.008779
Unnamed: 0.1                           -0.013705
Unnamed: 0                            

### Preprocessing

In [28]:
X = df_2017.drop(['case:concept:name', 'Unnamed: 0','concept:name', 'EventOrigin',
       'EventID', 'case:LoanGoal', 'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID', 'predicted_time_to_next_event_seconds'], axis=1)  # Drop target and non-feature columns
y = df_2017['time_to_next_event_seconds']  # Target for prediction

# Split data into training, validation, and test sets (60%, 20%, 20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)


In [32]:
# Preprocessing for numerical and categorical columns

# timing
start_time = time.time()


numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ])

# Fit the preprocessor on the training data and transform both training and validation data
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

# Initialize the model
model = XGBRegressor(objective='reg:squarederror')

# Fit the model using the transformed training set and validate using the transformed validation set
model.fit(X_train_transformed, y_train, eval_set=[(X_val_transformed, y_val)], early_stopping_rounds=10, verbose=False)

# Transform the test set and predict
X_test_transformed = preprocessor.transform(X_test)
y_pred_test = model.predict(X_test_transformed)

end_time = time.time()
# print the time for how long model runs
elapsed_time = end_time - start_time

print(f"Running time for model: {elapsed_time} seconds")

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'Test RMSE: {rmse_test}')

r2_score_value = r2_score(y_test, y_pred_test)
print(f'R² score: {r2_score_value}')
# For predicting and adding new columns, you need to transform the whole dataset
X_transformed = preprocessor.transform(X)

df_2017['predicted_time_to_next_event_seconds_XG'] = np.abs(model.predict(X_transformed).round(1))

df_2017['predicted_time_to_next_event_seconds_XG'] = df_2017['predicted_time_to_next_event_seconds_XG'].apply(lambda x: timedelta(seconds=x))
df_2017['time_to_next_event_seconds'] = df_2017['time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))
df_2017['predicted_time_to_next_event_seconds'] = df_2017['predicted_time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))

/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/Process_Mining_1/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Running time for model: 1.9336247444152832 seconds
Test RMSE: 21443.468759740736
R² score: 0.9983708948487269


TypeError: unsupported type for timedelta seconds component: Timedelta

In [44]:
df_2017 = df_2017.drop(['elapsed_time_from_start', 'predicted_start_time', 'predicted_step', 'timestamp_difference', 'event_seq', 'Unnamed: 0.1', 'Unnamed: 0', 'time_to_next_event'], axis=1, errors='ignore')

In [40]:
df_2017['concept:name next'] = le.inverse_transform(df_2017['concept:name next'])
df_2017['concept:name next rfc'] = le.inverse_transform(df_2017['concept:name next rfc'])
df_2017['concept:name'] = le.inverse_transform(df_2017['concept:name'].astype(int))

In [45]:
df_2017

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_to_next_event_seconds,predicted_time_to_next_event_seconds,concept:name next,concept:name next rfc,predicted_time_to_next_event_seconds_XG
0,1.0,0.0,W_Handle leads,2.0,701700.0,6.0,0.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:35.170000,W_Complete application,W_Complete application,0 days 00:00:51.500000
1,0.0,0.0,W_Complete application,2.0,630427.0,3.0,1.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:00.010000,A_Concept,A_Concept,0 days 00:01:30.300003
2,4.0,0.0,A_Concept,0.0,169678.0,1.0,2.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,4 days 23:34:08.080000,2 days 01:50:47.740000,A_Create Application,A_Create Application,4 days 22:36:52.093750
3,0.0,0.0,A_Create Application,0.0,219155.0,1.0,3.0,7.0,1.0,11273.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.050000,0 days 00:00:00.040000,A_Submitted,A_Submitted,0 days 00:00:42.099998
4,4.0,0.0,A_Submitted,0.0,130912.0,1.0,4.0,7.0,1.0,11273.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.190000,0 days 00:00:00.300000,W_Handle leads,W_Handle leads,0 days 00:00:58.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201081,4.0,148.0,A_Pending,0.0,47129.0,1.0,1200966.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00,0 days 00:00:00,W_Validate application,W_Validate application,0 days 00:00:35.500000
1201082,1.0,148.0,W_Validate application,2.0,708412.0,0.0,1200967.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:00.020000,O_Cancelled,W_Validate application,0 days 00:00:11.300000
1201083,4.0,148.0,O_Cancelled,1.0,323678.0,1.0,1200968.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,5323.0,0 days 00:00:00.010000,0 days 00:00:00.010000,O_Cancelled,W_Validate application,0 days 00:00:11.300000
1201084,4.0,148.0,O_Cancelled,1.0,250415.0,1.0,1200969.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,10669.0,0 days 00:00:00.010000,0 days 00:00:00.010000,O_Cancelled,W_Validate application,0 days 00:00:11.300000


In [46]:
df_2017.rename(columns={'time_to_next_event_seconds': 'time_to_next_event'}, inplace=True)
df_2017.rename(columns={'predicted_time_to_next_event_seconds': 'time_to_next_event_baseline'}, inplace=True)
df_2017.rename(columns={'predicted_time_to_next_event_seconds_XG': 'time_to_next_xgboost'}, inplace=True)


In [47]:
df_2017

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_to_next_event,time_to_next_event_baseline,concept:name next,concept:name next rfc,time_to_next_xgboost
0,1.0,0.0,W_Handle leads,2.0,701700.0,6.0,0.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:35.170000,W_Complete application,W_Complete application,0 days 00:00:51.500000
1,0.0,0.0,W_Complete application,2.0,630427.0,3.0,1.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:00.010000,A_Concept,A_Concept,0 days 00:01:30.300003
2,4.0,0.0,A_Concept,0.0,169678.0,1.0,2.0,5.0,1.0,25893.0,...,244.52,1.0,0.0,15000.0,1.0,4 days 23:34:08.080000,2 days 01:50:47.740000,A_Create Application,A_Create Application,4 days 22:36:52.093750
3,0.0,0.0,A_Create Application,0.0,219155.0,1.0,3.0,7.0,1.0,11273.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.050000,0 days 00:00:00.040000,A_Submitted,A_Submitted,0 days 00:00:42.099998
4,4.0,0.0,A_Submitted,0.0,130912.0,1.0,4.0,7.0,1.0,11273.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.190000,0 days 00:00:00.300000,W_Handle leads,W_Handle leads,0 days 00:00:58.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201081,4.0,148.0,A_Pending,0.0,47129.0,1.0,1200966.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00,0 days 00:00:00,W_Validate application,W_Validate application,0 days 00:00:35.500000
1201082,1.0,148.0,W_Validate application,2.0,708412.0,0.0,1200967.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,1.0,0 days 00:00:00.010000,0 days 00:00:00.020000,O_Cancelled,W_Validate application,0 days 00:00:11.300000
1201083,4.0,148.0,O_Cancelled,1.0,323678.0,1.0,1200968.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,5323.0,0 days 00:00:00.010000,0 days 00:00:00.010000,O_Cancelled,W_Validate application,0 days 00:00:11.300000
1201084,4.0,148.0,O_Cancelled,1.0,250415.0,1.0,1200969.0,11.0,1.0,25925.0,...,244.52,1.0,0.0,15000.0,10669.0,0 days 00:00:00.010000,0 days 00:00:00.010000,O_Cancelled,W_Validate application,0 days 00:00:11.300000


In [48]:
df_2017.to_csv('BPI_2017_rfc_xgboost.csv', index=False)